In [ ]:
# 목표: 서울시 홈페이지에 접속해서 서울소식을 1-5page를 크롤링해서 소식제목, 소식접속url, 담당부서, 게시일 수집 후 시각화하기

#과정
# 서울소식 버튼 클릭
# 게시글 크롤링
# 1-5 페이지
# 각 페이지 내 게시글 제목& 게시글 클릭 이동 페이지 url
# url 이동 후 게시글 업로드한 부서 & 게시날짜 
# 어느 부서가 가장 많은 게시물을 업로드했는지 (시각화)
# 어느 날짜에 게시물들이 많이 업로드되었는지 (시각화)



In [18]:
from selenium import webdriver #-> 크롤러 역할
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
#-----------------------------------------------------
from selenium.webdriver.common.by import By  #-> 명령어 수행 클래스
from selenium.webdriver.support.ui import WebDriverWait   #--> WebDriverWait:  기다리되, 그전에 실행준비가 끝나면 실행해라
from selenium.webdriver.support import expected_conditions as EC #--> expected_conditions을 EC라고 하자.
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd #-> 행과 열의 테이블을 만든다
from urllib.parse import urljoin #-> 한쪽은 상대경로, 한쪽은 절대경로를 가지는 경우, 안전하게 서로를 보완해서 하나의 url로 만들어라 


service = Service(ChromeDriverManager().install())
options = Options()
#options.add_argument("--headless") 
#options.add_argument("--disable-gpu") 
options.add_argument("--disable-dev-shm-usage") 
options.add_argument("--start-maximized") 
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36") 
options.add_argument("--lang=ko_KR") 

driver = webdriver.Chrome(service=service, options=options) #-> 첫번째 service는 chrome의 속성값으로 내재되어있는 정의되어있는 속성. 그 속성에 2번째 service라는 변수를 할당하는 것.
wait = WebDriverWait(driver, 10) #--> 크롤러에게 10초를 기다리되, 그전에 준비가 되면 바로 실행해라 명령

#리펙토리1) 목록 제목& 링크값을 수집하는 독립적인 함수 선언(=정의:definition) 
def collect_list_links_on_current_page() : #--> def-return과 짝꿍
    base = driver.current_url #https://www.seoul.go.kr/realmnews/in/list.do -> 서울시사이트 "새소식"페이지 링크가 베이스 
    items = driver.find_elements(By.CSS_SELECTOR, "div.news-lst div.item > a") #-> 값을 실제로 찾아와서 items 안에 넣어라

    links = []
    for item in items :
        title = item.find_element(By.CSS_SELECTOR, "em.subject").text.strip()
        href = item.get_attribute("href") #https://news.seoul.go.kr/welfare/archives/576396
        url = urljoin(base, href)
        links.append({"list_title":title, "url":url})
    return links
    
# 리팩토리2) 각 페이지당 제목, 부서명, 작성날짜를 찾아오는 독립적 함수선언
def scrap_detail_fields(article_url) : 
    driver.get(article_url)

    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#view_top")))
    title = driver.find_element(By.CSS_SELECTOR, "#view_top h3").text.strip()
    
    #담당부서
    dept_spnas = driver.find_elements(By.CSS_SELECTOR, "#view_top dd.dept span") 
    #--> 리스트 [일자리정책과, 고용훈련팀]를 -> 문자열 "일자리정책과 - 고용훈련팀" 형태로 만들어
    dept = "-".join([s.text.strip() for s in dept_spnas if s.text.strip()])
    modified_date = driver.find_element(By.CSS_SELECTOR, "#view_top dd.date").text.strip()
    return title, dept, modified_date


    
try :
    driver.get("https://www.seoul.go.kr/main/index.jsp") #-> 크롤러가 공략할 주소
    time.sleep(2)

    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='/realmnews/in/list.do']"))).click()
    #--> css 선택자 : 클릭할 요소  #--> 바깥에 "썼으면 안쪽에는 '을 써야함.
    #==> 저 "서울소식"이라는 요소가 클릭할수 있는 상태가 될때까지 기다리다가 클릭해라
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.news-lst div.item > a"))) #-> 클릭 X -> 모든 요소가 뜰때까지 기다려라 

    max_page = 5
    collected_links = []

    for page in range(1, max_page + 1) :
        driver.execute_script("window,scrollTo(0, document.body.scrollHeight);") #->웹 브라우저를 스크롤하는 자바스크립터의 기능을 driver 너가 수행해//자바스크립트는 꼭 끝에 ;로 끝냄.
        #-> window 창을 0(좌상단)부터 화면 제일 끝까지 스크롤해라
        time.sleep(0.5)

        #함수 호출 지점 -> 그 함수가 수집한 값을 리스트에 들어가게하는 것.
        collected_links.extend(collect_list_links_on_current_page())

        #단계들이 잘 가고있는지 확인하는 목적
        print(f"[목록수집] {page}페이지 누적 링크 {len(collected_links)}개")

        if page == max_page :
            break #반복문 퇴장함수
            
        next_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"#paging_main ul.pagination a[data-page='{page + 1}']")))
        next_btn.click()  #-> 다음 페이지로 이동

        #다음페이지에 가면 스크롤바를 위로 올린다
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(0.3)
    tmp_df = pd.DataFrame(collected_links).drop_duplicates(subset=["url"]).reset_index(drop=True)  #-> 리스트에서 공통 속성을 자동으로 필드명으로 가지고 2차원의 테이블을 만들어주는 것
   #중복되는 값을 버려라(drop) #-> 값을 버리면 인덱스가 꼬이니까 drop=True인 인덱스 값을 리셋해라
    link_list = tmp_df["url"].to_list() #-> 그리고 각각의 url을 끌어와라

    rows = []
    for i, link in enumerate(link_list, start=1) :
        try : 
            title, dept, modified_date = scrap_detail_fields(link)
            rows.append({
                "title": title,
                "url": link,
                "dept": dept,
                "modified_date": modified_date
            })
            print(f"[상세수집]{i}/{len(link_list)} 완료")
        except Exception as e :   #--> 에러가 발생한다면 이렇게 해
            #다른건 몰라도 link는 나올테니까 링크말고 다른건 넘겨
            rows.append({
                "title": "",
                "url": link,
                "dept": "",
                "modified_date": "",
                "error": str(e)
            })
            print(f"[상세실패] {i}/{len(link_list)} {link} ->{e}")
        
        time.sleep(0.3)
    df = pd.DataFrame(rows)
    filename = "seoul_news_p1_to_p5_with_dept_date.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
            
    print(f"CSV파일 저장완료 : {filename}")
       
finally :
    driver.quit() 
#--> try의 모든요소가 끝나면 최종적으로 finally를 해라

[목록수집] 1페이지 누적 링크 10개
[목록수집] 2페이지 누적 링크 20개
[목록수집] 3페이지 누적 링크 30개
[목록수집] 4페이지 누적 링크 40개
[목록수집] 5페이지 누적 링크 50개
[상세수집]1/50 완료
[상세수집]2/50 완료
[상세수집]3/50 완료
[상세수집]4/50 완료
[상세수집]5/50 완료
[상세수집]6/50 완료
[상세수집]7/50 완료
[상세수집]8/50 완료
[상세수집]9/50 완료
[상세수집]10/50 완료
[상세수집]11/50 완료
[상세수집]12/50 완료
[상세수집]13/50 완료
[상세수집]14/50 완료
[상세수집]15/50 완료
[상세수집]16/50 완료
[상세수집]17/50 완료
[상세수집]18/50 완료
[상세수집]19/50 완료
[상세수집]20/50 완료
[상세수집]21/50 완료
[상세수집]22/50 완료
[상세수집]23/50 완료
[상세수집]24/50 완료
[상세수집]25/50 완료
[상세수집]26/50 완료
[상세수집]27/50 완료
[상세수집]28/50 완료
[상세수집]29/50 완료
[상세수집]30/50 완료
[상세수집]31/50 완료
[상세수집]32/50 완료
[상세수집]33/50 완료
[상세수집]34/50 완료
[상세수집]35/50 완료
[상세수집]36/50 완료
[상세수집]37/50 완료
[상세수집]38/50 완료
[상세수집]39/50 완료
[상세수집]40/50 완료
[상세수집]41/50 완료
[상세수집]42/50 완료
[상세수집]43/50 완료
[상세수집]44/50 완료
[상세수집]45/50 완료
[상세수집]46/50 완료
[상세수집]47/50 완료
[상세수집]48/50 완료
[상세수집]49/50 완료
[상세수집]50/50 완료
CSV파일 저장완료 : seoul_news_p1_to_p5_with_dept_date.csv
